In [4]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='INSERT_ID_HERE', project_access_token='INSERT_TOKEN_HERE')
pc = project.project_context

from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space({'token':'INSERT_TOKEN_HERE'})

# Classifying customer notes with Watson NLP

This notebook was created using the [sample notebooks](https://dataplatform.cloud.ibm.com/exchange/public/entry/view/636001e59902133a4a23fd89f010e4cb?context=wx) in the Gallery. In this example we score customer complaints using the models we trained in the *Classify_notes_model_build* notebook. 

In [5]:
import watson_nlp

from watson_core.data_model.streams.resolver import DataStreamResolver
from watson_core.toolkit import fileio
from watson_nlp.blocks.classification.svm import SVM

### 1. Load customer interaction notes for scoring

**Important: regenerate data access code for the file in your project. Make sure to change the dataframe name after generation to complaint_df**
The csv file name that we will use for scoring is *notes_scoring.csv*

In [ ]:
import pandas as pd

# load data set into a dataframe
buffer = wslib.load_data("notes_scoring.csv")

complaint_df = pd.read_csv(buffer)
complaint_df.head()

In [9]:
# Identifyer for the column that contains the customer notes
text_col = 'Consumer complaint narrative'

### 2. Load models

In [10]:
# Load the out-of-the box Watson NLP models that we will use in combination with our trained models

# Syntax Model
syntax_model = watson_nlp.load('syntax_izumo_en_stock')
# USE Embedding Model
use_model = watson_nlp.load('embedding_use_en_stock')

In [11]:
# Load the models we previously saved in the project for scoring
svm_model = watson_nlp.load(wslib.load_data('classify_notes_svm'))
ensemble_model = watson_nlp.load(wslib.load_data('classify_notes_ensemble'))

###  3. Invoke Scoring

In [12]:
# Function that will be invoked for scoring 
def predict_product(text):
    # run syntax model first
    syntax_result = syntax_model.run(text)
    # run SVM model on top of syntax result
    svm_preds = svm_model.run(use_model.run(syntax_result, doc_embed_style='raw_text'))
    
    predicted_svm = svm_preds.to_dict()["classes"][0]["class_name"]
    
    ensemble_preds = ensemble_model.run(text)
    predicted_ensemble = ensemble_preds.to_dict()["classes"][0]["class_name"]
    return (predicted_svm, predicted_ensemble)

In [ ]:
# Invoke scoring

predictions = complaint_df[text_col].apply(lambda text: predict_product(text))
predictions_df = pd.DataFrame.from_records(predictions, columns=('Predicted SVM', 'Predicted Ensemble'))

# Display the results (for testing only)
predictions_df.head()

In [ ]:
# Merge scoring results with the original dataset
result_df = complaint_df.merge(predictions_df, how='left', left_index=True, right_index=True)
result_df.head()

### 4. Save Scoring Results in the project

In [ ]:
# Write scoring results to the project

# let's assume you have the pandas DataFrame pandas_df which contains the data
# you want to save as a csv file
wslib.save_data("notes_scoring_results.csv", result_df.to_csv(index=False).encode())